In [1]:
import asyncio
from playwright.async_api import async_playwright
from typing import List, Dict
import re

SEARCH_URL = "https://twitter.com/search?q={query}&f=live"



In [3]:
import asyncio
from playwright.async_api import async_playwright
from typing import List, Dict

async def scrape_tweets(query: str, max_tweets: int = 10) -> List[Dict]:
    tweets_data = []

    async with async_playwright() as pw:
        browser = await pw.chromium.launch(headless=False)  # set True when stable
        context = await browser.new_context(viewport={"width": 1920, "height": 1080})
        page = await context.new_page()

        search_url = f"https://twitter.com/search?q={query}&f=live"
        await page.goto(search_url)

        tweets_collected = 0
        scrolls = 0
        max_scrolls = 30

        # Wait for first tweet to appear
        try:
            await page.wait_for_selector('article[role="article"]', timeout=15000)
        except:
            print("No tweets found")
            return tweets_data

        while tweets_collected < max_tweets and scrolls < max_scrolls:
            tweet_elements = await page.query_selector_all('article[role="article"]')

            for el in tweet_elements[tweets_collected:]:
                try:
                    # Get tweet text
                    text_el = await el.query_selector('div[data-testid="tweetText"]')
                    text = await text_el.inner_text() if text_el else ""

                    # Get author handle
                    handle_el = await el.query_selector('div[data-testid="User-Names"] span')
                    author = await handle_el.inner_text() if handle_el else ""

                    tweets_data.append({"author": author, "text": text})
                    tweets_collected += 1
                    if tweets_collected >= max_tweets:
                        break
                except Exception:
                    continue

            # Scroll to load more
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await asyncio.sleep(2)
            scrolls += 1

        await browser.close()
    return tweets_data

In [4]:
def parse_tweet(data: Dict) -> Dict:
    """Parse raw tweet JSON into simplified dictionary."""
    result = jmespath.search(
        """{
        created_at: legacy.created_at,
        text: legacy.full_text,
        user_id: legacy.user_id_str,
        id: legacy.id_str
    }""",
        data
    )
    return result

In [5]:
tweets = await scrape_tweets("Brazilian favela", max_tweets=10)
for t in tweets:
    print(t)

No tweets found


In [6]:
await main()

NameError: name 'main' is not defined